asyncio
---
asyncio是Python 3.4版本引入的标准库，直接内置了对异步IO的支持。

asyncio的编程模型就是一个**消息循环**。我们从asyncio模块中直接获取一个**EventLoop**的引用，然后把需要执行的协程扔到EventLoop中执行，就实现了异步IO。


In [27]:
# 用asyncio实现Hello world
import asyncio,sys

@asyncio.coroutine
def hello():
    print("Hello world!")
    # 异步调用asyncio.sleep(1):
    r = yield from asyncio.sleep(1)
    print("Hello again!")

# 获取EventLoop:
#loop = asyncio.get_event_loop()
loop = asyncio.new_event_loop()
asyncio.set_event_loop(loop)  
# 执行coroutine
loop.run_until_complete(hello())
loop.close()


Hello world!
Hello again!


@asyncio.coroutine把一个**generator**标记为coroutine类型，然后，我们就把这个coroutine扔到EventLoop中执行。

hello()会首先打印出Hello world!，然后，**yield from**语法可以让我们方便地**调用另一个generator**。由于asyncio.sleep()也是一个coroutine，所以线程不会等待asyncio.sleep()，而是直接中断并执行下一个消息循环。当asyncio.sleep()返回时，线程就可以从yield from拿到返回值（此处是None），然后接着执行下一行语句。

把asyncio.sleep(1)看成是一个耗时1秒的IO操作，在此期间，主线程并未等待，而是去执行EventLoop中其他可以执行的coroutine了，因此可以实现并发执行。

我们用Task封装两个coroutine试试：

In [28]:
import threading
import asyncio

@asyncio.coroutine
def hello():
    print('Hello world! (%s)' % threading.currentThread())
    yield from asyncio.sleep(1)
    print('Hello again! (%s)' % threading.currentThread())

#loop = asyncio.get_event_loop()
loop = asyncio.new_event_loop()
asyncio.set_event_loop(loop)  
tasks = [hello(), hello()]
loop.run_until_complete(asyncio.wait(tasks))
loop.close()

Hello world! (<_MainThread(MainThread, started 140736493695936)>)
Hello world! (<_MainThread(MainThread, started 140736493695936)>)
Hello again! (<_MainThread(MainThread, started 140736493695936)>)
Hello again! (<_MainThread(MainThread, started 140736493695936)>)


由打印的当前线程名称可以看出，两个coroutine是由同一个线程并发执行的。

如果把asyncio.sleep()换成真正的IO操作，则多个coroutine就可以由一个线程并发执行。

In [ ]:
# 用asyncio的异步网络连接来获取sina、sohu和163的网站首页：
import asyncio

@asyncio.coroutine
def wget(host):
    print('wget %s...' % host)
    connect = asyncio.open_connection(host, 80)
    reader, writer = yield from connect
    header = 'GET / HTTP/1.0\r\nHost: %s\r\n\r\n' % host
    writer.write(header.encode('utf-8'))
    yield from writer.drain()
    while True:
        line = yield from reader.readline()
        if line == b'\r\n':
            break
        print('%s header > %s' % (host, line.decode('utf-8').rstrip()))
    # Ignore the body, close the socket
    writer.close()

# loop = asyncio.get_event_loop()
loop = asyncio.new_event_loop()
asyncio.set_event_loop(loop)  
tasks = [wget(host) for host in ['www.163.com', 'www.sina.com.cn', 'www.sohu.com']]
loop.run_until_complete(asyncio.wait(tasks))
loop.close()

RuntimeError: Event loop is closed
---
https://stackoverflow.com/questions/37778019/aiohttp-asyncio-runtimeerror-event-loop-is-closed

Methods run_until_complete, run_forever, run_in_executor, create_task, call_at explicitly check the loop and throw exception if it's closed.

Unless you have some(good) reasons, you might simply **omit the close** line:

In [ ]:
def scrap(category, field, pages, search, use_proxy, proxy_file):
    #...
    loop = asyncio.get_event_loop()

    to_do = [ get_pages(url, params, conngen) for url in urls ]
    wait_coro = asyncio.wait(to_do)
    res, _ = loop.run_until_complete(wait_coro)
    #...
    # loop.close()
    return [ x.result() for x in res ]

If you want to have each time a **brand new loop**, you have t create it **manually** and set as **default**:

In [ ]:
def scrap(category, field, pages, search, use_proxy, proxy_file):
    #...
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)    
    to_do = [ get_pages(url, params, conngen) for url in urls ]
    wait_coro = asyncio.wait(to_do)
    res, _ = loop.run_until_complete(wait_coro)
    #...
    return [ x.result() for x in res ]

A Curious Course on Coroutines and Concurrency
---
http://www.dabeaz.com/coroutines/index.html

A Web Crawler With asyncio Coroutines
---
http://aosabook.org/en/500L/a-web-crawler-with-asyncio-coroutines.html

异步IO（ asyncio） 协程
---
http://python.jobbole.com/87310/